In [1]:
import numpy as np
import scipy.io as sio
import os
import time
import spectral

from sklearn import cluster, metrics
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from tensorflow import keras

def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = np.zeros((X.shape[0] + 2*margin, X.shape[1] + 2*margin, X.shape[2]))
    zeroPaddedX[margin:X.shape[0] + margin, margin:X.shape[1] + margin, :] = X
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patchesData[patchIndex, :, :, :] = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0] - 1
    return patchesData, patchesLabels

def count_FLOPs(model):
    FLOPs = 0
    for i in range(len(model.layers)):
        layer = model.layers[i]
        if layer.__class__.__name__ == 'Conv3D':
            FLOPs += 2*np.prod(layer.kernel_size[:3])*np.prod(layer.output_shape[1:5])
        if layer.__class__.__name__ == 'Conv2D':
            FLOPs += 2*np.prod(layer.kernel_size[:2])*layer.input_shape[3]*np.prod(layer.output_shape[1:4])
        if layer.__class__.__name__ in ('MaxPooling3D', 'AveragePooling3D'):
            FLOPs += np.prod(layer.pool_size[:3])*np.prod(layer.output_shape[1:5])
        if layer.__class__.__name__ in ('MaxPooling2D', 'AveragePooling2D'):
            FLOPs += np.prod(layer.pool_size[:2])*np.prod(layer.output_shape[1:4])
        if layer.__class__.__name__ == 'GlobalAveragePooling3D':
            FLOPs += np.prod(layer.input_shape[1:5])
        if layer.__class__.__name__ == 'GlobalAveragePooling2D':
            FLOPs += np.prod(layer.input_shape[1:4])             
        if layer.__class__.__name__ in ('Activation', 'Add', 'Multiply'):
            FLOPs += np.prod(layer.output_shape[1:])            
        if layer.__class__.__name__ == 'Dense':
            FLOPs += 2*layer.input_shape[1]*layer.output_shape[1]          
    return FLOPs

In [2]:
test_ratio = 0.9
patch = 25
PCsNum = 15
ClusterNum = 100  
HSI = sio.loadmat(os.path.join(os.getcwd(),'data/Indian_pines_corrected.mat'))['indian_pines_corrected']
GT = sio.loadmat(os.path.join(os.getcwd(),'data/Indian_pines_gt.mat'))['indian_pines_gt']
HPCs = PCA(n_components=PCsNum, whiten=True).fit_transform(HSI.reshape(-1, HSI.shape[2])).reshape(HSI.shape[0], HSI.shape[1], PCsNum)

In [3]:
start_time = time.process_time()
clusterstack = np.zeros(HPCs.shape)

mbk_means = cluster.MiniBatchKMeans(n_clusters=ClusterNum).fit(HPCs.reshape((-1, HPCs.shape[2])))
clusterlabel = mbk_means.labels_.reshape((GT.shape))

for i in range(ClusterNum):
    clusterstack[clusterlabel==i]=np.mean(HPCs[clusterlabel==i], axis=0)
    
timeused = (time.process_time() - start_time)
print('Shape of Clusters:', clusterstack.shape, 'Clustering Time: ', timeused, 's.  # of Clusters:', np.max(clusterlabel)+1)

Shape of Clusters: (145, 145, 15) Clustering Time:  2.25 s.  # of Clusters: 100


In [4]:
X, y = createImageCubes(clusterstack, GT, windowSize=patch)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_ratio, random_state=345,stratify=y)
ytrain = keras.utils.to_categorical(ytrain)

## input layer
IL = keras.Input(shape=(patch, patch, PCsNum), name='IL')
C1 = keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu', name='C1')(IL)
P1 = keras.layers.MaxPool2D()(C1)
C2 = keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='C2')(P1) 
P2 = keras.layers.MaxPool2D()(C2)
## flatten
FL = keras.layers.Flatten(name='FL')(P2)        
## fully connected layers
D1 = keras.layers.Dense(units=128, activation='relu', name='D1')(FL)
D1 = keras.layers.Dropout(0.4, name='dropout1')(D1)
D2 = keras.layers.Dense(units=64, activation='relu', name='D2')(D1)
D2 = keras.layers.Dropout(0.4, name='dropout2')(D2)
OL = keras.layers.Dense(units=ytrain.shape[1], activation='softmax', name='OL')(D2)
        
model = keras.models.Model(inputs=IL, outputs=OL, name='2DCNN_FCs')
        
# compiling the model
adam = keras.optimizers.Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()
print('FLOPs:', format(count_FLOPs(model), ','))

Model: "2DCNN_FCs"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
IL (InputLayer)              [(None, 25, 25, 15)]      0         
_________________________________________________________________
C1 (Conv2D)                  (None, 21, 21, 64)        24064     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
C2 (Conv2D)                  (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
FL (Flatten)                 (None, 1024)              0         
_________________________________________________________________
D1 (Dense)                   (None, 128)               13

In [5]:
start_time = time.process_time()
# training
model.fit(x=Xtrain, y=ytrain, batch_size=256, epochs=100, verbose=0)
training_time = time.process_time()-start_time

# predict
y_pred = np.argmax(model.predict(Xtest), axis=1)
oa = metrics.accuracy_score(ytest, y_pred)
kappa = metrics.cohen_kappa_score(ytest, y_pred)

print('ClusterCNN:', oa, kappa, ' Training Time:', training_time)

ClusterCNN: 0.9841734417344173 0.9819535802429964  Training Time: 25.703125
